In [12]:
from math import sqrt
from random import randint,shuffle
import random
import time

# Defining infinity
INF = 1e18
mult = 1e6
 
# Structure to represent a 2D point
class Point :
    def __init__(self,X=0,Y=0) -> None:
        self.X=X
        self.Y=Y
        
# Structure to represent a 2D circle
class Circle :
    def __init__(self,c=Point(),r=0) -> None:       
        self.C=c
        self.R=r
 
 
# Function to return the euclidean distance
# between two points
def dist(a, b):
    return sqrt(pow(a.X - b.X, 2)
                + pow(a.Y - b.Y, 2))

# Function to check whether a point lies inside
# or on the boundaries of the circle
def is_inside(c, p):
    return dist(c.C, p) <= c.R
 
# The following two functions are used
# To find the equation of the circle when
# three points are given.
 
# Helper method to get a circle defined by 3 points
def get_circle_center(bx, by,
                        cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D),
             (bx * C - cx * B) / (2 * D))
 
# Function to return the smallest circle
# that intersects 2 points
def circle_from1(A, B):
    # Set the center to be the midpoint of A and B
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0 )
 
    # Set the radius to be half the distance AB
    return Circle(C, dist(A, B) / 2.0 )
 
# Function to return a unique circle that
# intersects three points
def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y,
                                C.X - A.X, C.Y - A.Y)
 
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))
 
 
 
 
 
# Function to check whether a circle
# encloses the given points
def is_valid_circle(c, P):
 
    # Iterating through all the points
    # to check  whether the points
    # lie inside the circle or not
    for p in P:
        if (not is_inside(c, p)):
            return False
    return True
 
# Function to return the minimum enclosing
# circle for N <= 3
def min_circle_trivial(P):
    assert(len(P) <= 3)
    if not P :
        return Circle()
     
    elif (len(P) == 1) :
        return Circle(P[0], 0)
     
    elif (len(P) == 2) :
        return circle_from1(P[0], P[1])
     
 
    # To check if MEC can be determined
    # by 2 points only
    for i in range(3):
        for j in range(i + 1,3):
 
            c = circle_from1(P[i], P[j])
            if (is_valid_circle(c, P)):
                return c
         
     
    return circle_from2(P[0], P[1], P[2])
 
# Returns the MEC using Welzl's algorithm
# Takes a set of input points P and a set R
# points on the circle boundary.
# n represents the number of points in P
# that are not yet processed.
def welzl_helper(P, R, n):
    # Base case when all points processed or |R| = 3
    if (n == 0 or len(R) == 3) :
        return min_circle_trivial(R)
     
 
    # Pick a random point randomly
    idx = randint(0,n-1)
    p = P[idx]
 
    # Put the picked point at the end of P
    # since it's more efficient than
    # deleting from the middle of the vector
    P[idx], P[n - 1]=P[n-1],P[idx]
 
    # Get the MEC circle d from the
    # set of points P - :p
    d = welzl_helper(P, R.copy(), n - 1)
 
    # If d contains p, return d
    if (is_inside(d, p)) :
        return d
     
 
    # Otherwise, must be on the boundary of the MEC
    R.append(p)
 
    # Return the MEC for P - :p and R U :p
    return welzl_helper(P, R.copy(), n - 1)

def minimum_enclosing_circle(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))

if __name__=='__main__':
    alt = 1000                     
    lat_position = 24.788090       
    lon_position = 120.997858      
    TxR = 300
    lat = [0]*500
    lon = [0]*500
    laspeed = [0]*500
    lospeed = [0]*500
    lospeed_total = 0
    laspeed_total = 0
    latmin = 24.7890
    latmax = 24.7895
    lonmin = 120.9975
    lonmax = 120.9980
    smax = 200*0.000009
    smin = 20*0.000009
    user = 100
    count_out = 0
    count_num = 0
    time_max = 0
    ladiff_total = 0
    lodiff_total = 0
    count_lad = 0
    count_lod = 0
    valid = [1]*500
    validuser = user
    
    
    
    f= open("users_6G.txt","r")
    while True:
        laspeed_total = 0
        lospeed_total = 0
        count_num += 1
        start = time.time()
        print("Current valid users number = ",validuser)
        for i in range(user):
            line = f.readline()
            lat[i] = float(line)
            line = f.readline()
            lon[i] = float(line)
            line = f.readline()
            laspeed[i] = float(line)
            line = f.readline()
            lospeed[i] = float(line)
            
            
        v1 = []
        for i in range(0, user):
            if valid[i] == 1:
                v1.append(Point(lon[i]*mult,lat[i]*mult))
        mec = minimum_enclosing_circle(v1)
        
        for i in range(0,user):
            if valid[i] == 1:
                lospeed_total += lospeed[i];
                laspeed_total += laspeed[i];
        lospeed_uav = lospeed_total/validuser;
        laspeed_uav = laspeed_total/validuser;

        
        if mec.R/mult > TxR*0.000009:
            print("Out of range")
            for i in range(user-1 , -1, -1): # 99~0
                v2 = []
                for j in range(user-1 , i-1, -1): #user-i
                    if valid[j] == 1:
                        v2.append(Point(lon[j]*mult,lat[j]*mult))
                mec2 = minimum_enclosing_circle(v2);
                if mec2.R/mult > TxR*0.000009:
                    v3 = []
                    for j in range(user-1 , i ,-1):
                        if valid[j] == 1:
                            v3.append(Point(lon[j]*mult,lat[j]*mult))
                    mec3 = minimum_enclosing_circle(v3)
                    for m in range(user):
                        if sqrt(pow(lat[m] - mec3.C.Y/mult, 2) + pow(lon[m] - mec3.C.X/mult, 2)) > TxR * 0.000009:
                            if valid[m] == 1:
                                print("Kick off user ",m)
                                valid[m] = 0
                    cnt = 0
                    for k in range(user):
                        if valid[k] == 1:
                            cnt += 1
                    validuser = cnt
                    if mec3.C.Y/mult > lat_position:
                        print("前移",(mec3.C.Y/mult - lat_position) / 0.000009,"公尺")
                    else:
                        print("後移",(lat_position - mec3.C.Y/mult) / 0.000009,"公尺")
                    if mec3.C.X/mult > lon_position:
                        print("左移",(mec3.C.X/mult - lon_position) / 0.000009,"公尺")
                    else:
                        print("右移",(lon_position - mec3.C.X/mult) / 0.000009,"公尺")
                    lat_position = mec3.C.Y/mult
                    lon_position = mec3.C.X/mult
                    break
        elif mec.R/mult == 0:
            print("mec.Y = %5.6f" %mec.C.Y/mult)
            print("mec.X = %5.6f" %mec.C.X/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
           
        else:
            print("lat = ",mec.C.Y/mult)
            print("lon = ",mec.C.X/mult)
            #print("R = ",mec.R/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            print("real laspeed = ",abs(lat_position - mec.C.Y/mult) / 0.000009/0.03,"m/s")
            print("real lospeed = ",abs(lon_position - mec.C.X/mult) / 0.000009/0.03,"m/s")
            print("predict laspeed = ",laspeed_uav/0.03,"m/s")
            print("predict lospeed = ",lospeed_uav/0.03,"m/s")
            larspd = abs((lat_position - mec.C.Y/mult) / 0.000009/0.03)
            lorspd = abs((lon_position - mec.C.X/mult) / 0.000009/0.03)
            lapspd = abs(laspeed_uav/0.03)
            lopspd = abs(lospeed_uav/0.03)
            if larspd != 0:
                ladiff = abs((larspd - lapspd)/larspd)
            if lorspd != 0:
                lodiff = abs((lorspd - lopspd)/lorspd)
            
            if count_num > 1:
                ladiff_total = ladiff_total + ladiff
                lodiff_total = lodiff_total + lodiff
                ladiff_mean = ladiff_total/(count_num-1)
                lodiff_mean = lodiff_total/(count_num-1)
                if ladiff > 0.2:
                    count_lad += 1
                if lodiff > 0.2:
                    count_lod += 1
                print("la誤差率 = ", ladiff_mean)
                print("lo誤差率 = ", lodiff_mean)
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
            
        end = time.time()
        t = end - start
        if t < 0.03:
            time.sleep(0.03-t)

Current valid users number =  100
lat =  24.791502241946663
lon =  120.99975898008695
前移 379.1379940736306 公尺
左移 211.22000966203004 公尺
real laspeed =  12637.933135787687 m/s
real lospeed =  7040.666988734335 m/s
predict laspeed =  20.128854999999998 m/s
predict lospeed =  19.87035133333334 m/s
finish
Current valid users number =  100
lat =  24.791508539009588
lon =  120.99976654859931
前移 0.6996736582702093 公尺
左移 0.8409458176730469 公尺
real laspeed =  23.322455275673644 m/s
real lospeed =  28.031527255768232 m/s
predict laspeed =  20.06261533333333 m/s
predict lospeed =  20.000838666666674 m/s
la誤差率 =  0.13977258842641974
lo誤差率 =  0.286487729185324
finish
Current valid users number =  100
lat =  24.791516062095276
lon =  120.99977454754556
前移 0.8358984098480151 公尺
左移 0.8887718049916454 公尺
real laspeed =  27.863280328267173 m/s
real lospeed =  29.62572683305485 m/s
predict laspeed =  20.127697666666663 m/s
predict lospeed =  19.86760733333334 m/s
la誤差率 =  0.20869950232196216
lo誤差率 =  0.30

lat =  24.791648
lon =  120.999939
前移 0.49999999992185923 公尺
左移 0.7222222219777702 公尺
real laspeed =  16.666666664061975 m/s
real lospeed =  24.074074065925675 m/s
predict laspeed =  20.151301999999998 m/s
predict lospeed =  19.981973000000007 m/s
la誤差率 =  0.17536338909430668
lo誤差率 =  0.24544878032854459
finish
Current valid users number =  100
lat =  24.7916525
lon =  120.999945
前移 0.5000000003166052 公尺
左移 0.666666666562479 公尺
real laspeed =  16.666666677220174 m/s
real lospeed =  22.222222218749298 m/s
predict laspeed =  20.116724666666663 m/s
predict lospeed =  19.87454833333334 m/s
la誤差率 =  0.17658031563820692
lo誤差率 =  0.240071724348994
finish
Current valid users number =  100
lat =  24.791657
lon =  120.999952
前移 0.49999999992185923 公尺
左移 0.7777777773930615 公尺
real laspeed =  16.666666664061975 m/s
real lospeed =  25.925925913102052 m/s
predict laspeed =  20.205762666666665 m/s
predict lospeed =  19.830823000000006 m/s
la誤差率 =  0.177904961732698
lo誤差率 =  0.23988746888290194
finish

lat =  24.79176
lon =  121.0000945
前移 0.5555555557318964 公尺
左移 0.7222222219777702 公尺
real laspeed =  18.518518524396548 m/s
real lospeed =  24.074074065925675 m/s
predict laspeed =  20.16432833333333 m/s
predict lospeed =  19.95948633333334 m/s
la誤差率 =  0.17322272010421597
lo誤差率 =  0.20679210528827285
finish
Current valid users number =  100
lat =  24.7917645
lon =  121.0001015
前移 0.49999999992185923 公尺
左移 0.7777777773930615 公尺
real laspeed =  16.666666664061975 m/s
real lospeed =  25.925925913102052 m/s
predict laspeed =  20.076964666666665 m/s
predict lospeed =  19.918149666666675 m/s
la誤差率 =  0.17385062330589685
lo誤差率 =  0.20729083343204993
finish
Current valid users number =  100
lat =  24.7917695
lon =  121.0001075
前移 0.5555555557318964 公尺
左移 0.666666666562479 公尺
real laspeed =  18.518518524396548 m/s
real lospeed =  22.222222218749298 m/s
predict laspeed =  20.120686999999997 m/s
predict lospeed =  19.847899333333338 m/s
la誤差率 =  0.17213820123431303
lo誤差率 =  0.20532129806790025
f

lat =  24.791895
lon =  121.000283
前移 0.5555555557318964 公尺
左移 0.7222222219777702 公尺
real laspeed =  18.518518524396548 m/s
real lospeed =  24.074074065925675 m/s
predict laspeed =  20.14015733333333 m/s
predict lospeed =  19.90198333333334 m/s
la誤差率 =  0.1729661314550543
lo誤差率 =  0.19379356531331857
finish
Current valid users number =  100
lat =  24.791899
lon =  121.0002895
前移 0.44444444450656795 公尺
左移 0.7222222219777702 公尺
real laspeed =  14.8148148168856 m/s
real lospeed =  24.074074065925675 m/s
predict laspeed =  20.03363033333333 m/s
predict lospeed =  19.828431333333338 m/s
la誤差率 =  0.17523580127601543
lo誤差率 =  0.19357285521332124
finish
Current valid users number =  100
lat =  24.791904
lon =  121.000296
前移 0.5555555553371505 公尺
左移 0.7222222235567541 公尺
real laspeed =  18.51851851123835 m/s
real lospeed =  24.07407411855847 m/s
predict laspeed =  20.219284333333327 m/s
predict lospeed =  19.849767000000003 m/s
la誤差率 =  0.17419337069043067
lo誤差率 =  0.19334658475373787
finish
Cu

lo誤差率 =  0.19793868914970464
finish
Current valid users number =  99
lat =  24.792210810885003
lon =  121.00022860067136
前移 0.8031103170146202 公尺
左移 0.7038890019986057 公尺
real laspeed =  26.77034390048734 m/s
real lospeed =  23.46296673328686 m/s
predict laspeed =  20.294157575757573 m/s
predict lospeed =  19.839170707070707 m/s
la誤差率 =  0.1934015843699548
lo誤差率 =  0.1975359927453803
finish
Current valid users number =  99
lat =  24.79221701063032
lon =  121.00023531565124
前移 0.6888605908904487 公尺
左移 0.746108875028363 公尺
real laspeed =  22.962019696348293 m/s
real lospeed =  24.870295834278767 m/s
predict laspeed =  20.188176767676765 m/s
predict lospeed =  19.7975430976431 m/s
la誤差率 =  0.19273552715417316
lo誤差率 =  0.19759500502436272
finish
Current valid users number =  99
lat =  24.792233137569614
lon =  121.00023362903
前移 1.7918821437016654 公尺
右移 0.18740235911233563 公尺
real laspeed =  59.72940479005552 m/s
real lospeed =  6.246745303744521 m/s
predict laspeed =  20.293560606060606 m

...

lat =  24.797320503472637
lon =  121.00383266578945
前移 0.5323656628617199 公尺
左移 0.5132323145239247 公尺
real laspeed =  17.745522095390665 m/s
real lospeed =  17.107743817464158 m/s
predict laspeed =  21.81609814814815 m/s
predict lospeed =  18.41049506172839 m/s
la誤差率 =  0.2728802249646248
lo誤差率 =  0.18450326478411663
finish
Current valid users number =  54
lat =  24.797324030604745
lon =  121.00383767462576
前移 0.39190356757343475 公尺
左移 0.5565373678943312 公尺
real laspeed =  13.063452252447826 m/s
real lospeed =  18.551245596477706 m/s
predict laspeed =  21.846599382716047 m/s
predict lospeed =  18.75193518518518 m/s
la誤差率 =  0.27331775486600435
lo誤差率 =  0.18431302913740535
finish
Current valid users number =  54
lat =  24.79732885543216
lon =  121.00384229459448
前移 0.536091934893395 公尺
左移 0.5133298578306899 公尺
real laspeed =  17.86973116311317 m/s
real lospeed =  17.110995261022996 m/s
predict laspeed =  21.833061111111107 m/s
predict lospeed =  18.44852222222222 m/s
la誤差率 =  0.27326137

Current valid users number =  54
lat =  24.79742872355294
lon =  121.00395064141206
前移 0.4062656956528422 公尺
左移 0.5713131277692204 公尺
real laspeed =  13.54218985509474 m/s
real lospeed =  19.04377092564068 m/s
predict laspeed =  21.73335679012345 m/s
predict lospeed =  18.408698148148144 m/s
la誤差率 =  0.27412043145327536
lo誤差率 =  0.181264882091204
finish
Current valid users number =  54
lat =  24.797433564768053
lon =  121.00395527385531
前移 0.5379127902611142 公尺
左移 0.5147159167033047 公尺
real laspeed =  17.93042634203714 m/s
real lospeed =  17.157197223443493 m/s
predict laspeed =  21.742837037037035 m/s
predict lospeed =  18.73465987654321 m/s
la誤差率 =  0.2740547985734329
lo誤差率 =  0.18116955323631712
finish
Current valid users number =  54
lat =  24.79743792745612
lon =  121.0039602849498
前移 0.4847431185339701 公尺
左移 0.5567882763241666 公尺
real laspeed =  16.15810395113234 m/s
real lospeed =  18.559609210805554 m/s
predict laspeed =  21.867124074074074 m/s
predict lospeed =  18.73124938271

lat =  24.79753819330946
lon =  121.00406790029896
前移 0.5395852791739091 公尺
左移 0.5160014985171478 公尺
real laspeed =  17.986175972463638 m/s
real lospeed =  17.200049950571596 m/s
predict laspeed =  21.916489506172837 m/s
predict lospeed =  18.5666024691358 m/s
la誤差率 =  0.2749558227269106
lo誤差率 =  0.17814771279297495
finish
Current valid users number =  54
lat =  24.79754268164875
lon =  121.00407305062828
前移 0.498704365507289 公尺
左移 0.5722588127810013 公尺
real laspeed =  16.623478850242968 m/s
real lospeed =  19.075293759366712 m/s
predict laspeed =  21.744217901234563 m/s
predict lospeed =  18.40224629629629 m/s
la誤差率 =  0.2749902522185603
lo誤差率 =  0.17799905099745117
finish
Current valid users number =  54
lat =  24.797547907444013
lon =  121.00407818977239
前移 0.5806439181318032 公尺
左移 0.5710160119557864 公尺
real laspeed =  19.354797271060107 m/s
real lospeed =  19.033867065192883 m/s
predict laspeed =  21.836217283950617 m/s
predict lospeed =  18.472499382716045 m/s
la誤差率 =  0.274837670

Current valid users number =  54
lat =  24.79764841930452
lon =  121.00418678455917
前移 0.5399187558276076 公尺
左移 0.5943199899623627 公尺
real laspeed =  17.997291860920253 m/s
real lospeed =  19.810666332078757 m/s
predict laspeed =  21.766495679012344 m/s
predict lospeed =  18.597275925925924 m/s
la誤差率 =  0.27536254964341816
lo誤差率 =  0.1753508839691713
finish
Current valid users number =  54
lat =  24.797654023891408
lon =  121.00419238887562
前移 0.6227318765341907 公尺
左移 0.6227018284713873 公尺
real laspeed =  20.757729217806357 m/s
real lospeed =  20.75672761571291 m/s
predict laspeed =  21.816794444444444 m/s
predict lospeed =  18.68023395061728 m/s
la誤差率 =  0.27513479099744687
lo誤差率 =  0.17527442576087068
finish
Current valid users number =  54
lat =  24.79765834669018
lon =  121.00419735377986
前移 0.48031097479553053 公尺
左移 0.5516560261437714 公尺
real laspeed =  16.010365826517685 m/s
real lospeed =  18.388534204792382 m/s
predict laspeed =  21.906011111111113 m/s
predict lospeed =  18.450

ValueError: could not convert string to float: ''